# Tarea 2
Estudiantes:
- Matías Fuentes
- Larry Uribne

Carga de librerías e inicialización de sparkContext

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .getOrCreate()
sc = spark.sparkContext

23/06/21 11:42:06 WARN Utils: Your hostname, Matiass-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.100.162 instead (on interface en0)
23/06/21 11:42:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/21 11:42:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Ejemplo inicial

In [17]:
nodes = [1, 2, 3, 4]
edges = [(1, 2), (2, 3), (2, 4), (3, 2)]
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)

## Problema 1

1. Preparemos un RDD que contenga a cada nodo con su Page Rank inicial. El Page Rank inicial está dado por $\frac{1}{N_{nodos}}$

In [89]:
n_nodes = rdd_nodes.count()
# Disponibilizamos la cantidad de nodos para todos los workers
bc_n_nodes = sc.broadcast(n_nodes)
rdd_ini = rdd_nodes.map(lambda x: (x, 1/bc_n_nodes.value))
rdd_ini.collect()

[(1, 0.25), (2, 0.25), (3, 0.25), (4, 0.25)]

El mensaje saliente de cada nodo será el mensaje inicial dividido por la cantidad de vecinos

In [86]:
neigh_counts = rdd_edges.countByKey()
# Disponibilizamos la cantidad de mensajes enviados por nodo para todos los workers
bc_neigh_counts = sc.broadcast(neigh_counts)
neigh_counts

defaultdict(int, {1: 1, 2: 2, 3: 1})

Vemos que solo el nodo 2 tiene más de 1 vecino, por tanto el único mensaje que será modificado es el suyo.

In [87]:
def prepare_message(node, init_msg):
    if neigh_counts[node] != 0:
        message = init_msg / bc_neigh_counts.value[node]
        return message
    return init_msg

rdd_prep_msg = rdd_ini.map(lambda x: (x[0], prepare_message(x[0], x[1])))
rdd_prep_msg.collect()

[(1, 0.25), (2, 0.125), (3, 0.25), (4, 0.25)]

2. Función que envía los mensajes a los nodos correspondientes y se hace cargo del merge de los mensajes recibidos por cada nodo. Debe retornar un RDD que para cada nodo diga cuál es el mensaje final recibido.

Hacemos un join. Las llaves son los nodos emisores, y os valores resultantes seran tuplas donde el primer elemento es el nodo receptor y el segundo elemento es el mensaje recibido.

In [92]:
rdd_msg_sent = rdd_edges.join(rdd_prep_msg)
rdd_msg_sent.collect()

[(1, (2, 0.25)), (2, (3, 0.125)), (2, (4, 0.125)), (3, (2, 0.25))]

Luego, generamos otro RDD que contenga solo los valores de la operación anterior y aplicamos reduceByKey para sumar los mensajes

In [97]:
rdd_received_msg = rdd_msg_sent.values().reduceByKey(lambda x, y: x + y)
rdd_received_msg.collect()

[(2, 0.5), (3, 0.125), (4, 0.125)]